In [1]:
#importing libraries
import networkx as nx
import numpy as np
import pandas as pd
from collections import defaultdict
import functions_hw5 as fn

In [2]:
#Categories
#Here I created a dictionary with each category and the articles that contains
categories = {}
with open("wiki-topcats-categories.txt") as f:
    for line in f:
        (key, val) = line.lstrip('Category:').rstrip('\n').split(";")        
        keys = val.split(" ")[1:]
        #keep only the categories with more that 3500 articles
        if len(keys) >= 3500:
            categories[key] = val.split(" ")[1:]
            
#flatten list with all the articles contained in the categories that we are interested in            
elements = set([x for cat in categories for x in categories[cat]])

In [3]:
#EDGES
#here I created a list with all the edges of the graph
edges = []
with open("wiki-topcats-reduced.txt") as f:
    for line in f:
        a, b = line.rstrip('\n').split("\t")
        if a in elements and b in elements:
            edges.append((a,b))

In [4]:
#Names
#Here is a dictinoary with the name of every articles, That would be useful in the final print of the results,
#for now let's just work for the numbers-ID since it's easier
names = {}
with open("wiki-topcats-page-names.txt") as f:
    for line in f:
        (key, val) = line.rstrip('\n').split(" ",1)
        names[key] = val

In [5]:
#Graph creation
#Creation of graph and
G = nx.DiGraph()
G.add_edges_from(edges,weight=1)

#Assign name attribute to every node of the graph
for node in G.nodes():
    G.node[node]['name'] = names[node]
    
#Assing categories attribute to every node of the graph
#ATTENTION!!!Some nodes are belonging to more than one categories
for cat in categories:
    for n in categories[cat]:
        if G.has_node(n):
        #if the node is already belonging to a categore we append the new category to the category attribute
            if 'category' in G.node[n].keys():
                G.node[n]['category'].append(cat)
        #if the node is not on the other hand we create a list with a single element
            else:
                G.node[n]['category'] = [cat]
                  
    
#delete nodes in categories dictionary values that are not in the graph.
for cat in categories.keys():
    categories[cat] = [node for node in categories[cat] if node in G.nodes()]

In [6]:
E = G.number_of_edges() #number of edges    ## len(edges)
V = G.number_of_nodes() #number of nodes    ## len(set([y for x in Edges for y in x]))

#s = []
##for node in G.nodes():
#    s.append(G.degree[node])
#avg_node_degee = np.mean(s) #average node degree

In [7]:
print('G is directed Graph: ',G.is_directed())
print('number of edges: ',E)
print('number of nodes: ',V)
#since it is a directed graph:
#avg node degree of input nodes or predecessors
print('Average in-node/predecessors degree: ',float(nx.info(G).split('\n')[-2].split(" ")[-1]))
#avg node degree of output nodes or successors
print('Average out-node/successors degree: ',float(nx.info(G).split('\n')[-1].split(" ")[-1]))

G is directed Graph:  True
number of edges:  2645247
number of nodes:  461193
Average in-node/predecessors degree:  5.7357
Average out-node/successors degree:  5.7357


The [**Density**](https://en.wikipedia.org/wiki/Dense_graph) of a *directed* graph can be difined through this relation:

\begin{equation*}
\ \frac{E}{V(V-1)}\\
\end{equation*}

The maximum number of edges for an directed graph is V(V-1)  so the maximal density is 1 (for complete graphs) and the minimal density is 0 [(Coleman & Moré 1983)](https://epubs.siam.org/doi/10.1137/0720013).

In [8]:
print('The graph density is: ',E/(V*(V-1)))  #graph density

The graph density is:  1.2436602635647606e-05


* **NOT** very dense

# Block Ranking

In [9]:
#random input category
inp_cat = np.random.choice(list(categories.keys()),1)[0]

#for matter of speed we have choosen as an input category the smallest in size:
#inp_cat = 'Year_of_birth_unknown'

Here we will try to compute the Block Ranking vector such that we order the categories according to their distance from the input one. In order to compute the distance between each category $C_{i}$ and the input $C_{0}$ we will use the formula below:

\begin{equation*}
\ Distance(C_{0},C_{i})\; = \;median(\;shortest\;path(C_{0},C_{i})\;) \\
\end{equation*}


In order to compute the shortest path we will apply the [Breadth First Search](https://en.wikipedia.org/wiki/Breadth-first_search) algorithm (BFS) since is computing all the possible shortest paths from a starting node to all the possible ones much quicker than other ones like [Dijkstra](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm) in contrast.

The results of the respective function will have the follow format:

![](https://github.com/CriMenghini/ADM-2018/blob/master/Homework_5/imgs/sort_inside_categories.png?raw=true)

In [10]:
#block_ranking score of sorted categories to the input one.
block_ranking = fn.blockRanking(inp_cat,categories,G)

# Updating Categories containing unique elements

**Delete** elements existing in more than one categories and defining the unique nodes new_categories

we are doing that since we noticed that one article might belong to a single category or multiple ones. 
In the case it belongs to more than one:

 - If the article belongs to the input category it belongs to that one.
    
 - Otherwise, the category of the article will correspond, among the categories it belongs to, to the closest to the input category.

In [11]:
#defining a set with all the articles of the input category
seen = set(categories[inp_cat])
#dictionary creation with the new_categories updated since there are elements in only one category
#according to the rules above
#this dictionary will be SORTED accorting to the blockRanking vector
new_categories = {inp_cat:categories[inp_cat]}
for _,cat in block_ranking[1:]:
    c = []
    for art in categories[cat]:
        if art not in seen:
            seen.add(art)
            c.append(art)
    new_categories[cat] = c
    
#update the category attribute of the graph according to the updated new_category.  
for cat in new_categories:
    for n in new_categories[cat]:
            G.node[n]['category'] = cat

### **Block Ranking Vector**

In [12]:
#block_ranking printing
df = pd.DataFrame(list(block_ranking),columns=['Distance','Category'])
df

,Distance,Category
0,0.0,American_film_actors
1,4.0,American_Jews
2,4.0,American_films
3,4.0,American_television_actors
4,4.0,Article_Feedback_Pilot
5,4.0,English-language_films
6,5.0,American_military_personnel_of_World_War_II
7,5.0,Black-and-white_films
8,5.0,British_films
9,5.0,Debut_albums


# Articles Sorting according to the score

As final step now we will sort each aerticle inside every category. In order to do that we will follow the following illustration:

![](https://github.com/CriMenghini/ADM-2018/raw/master/Homework_5/imgs/algorithm.PNG)

*More specificly we have to compute for each subgraph the score of the nodes. Each time we have to create a subgraph consisted from the nodes of the nodes of all the previous categories and the nodes of the current category. If the predecessors of the node coming from the same category, they give to it score equal to one. On the other hand if they come from the previous categories other than the current they give to it score equal to their own score. This continues consequently until we create a total subgraph equal to the original one. Finally, we sort the nodes of each category according to their score from the greater to the smaller*

In [13]:
#calculating the score for each node
score = fn.scoring(new_categories,G)

In [14]:
#final ranking list
ranking = []

#for every category in the new_categories dictionary that is sorted according to the category distance
#we sort the articles according to the score dictionary
for cat in new_categories:
    s = []
    for art in new_categories[cat]:
        s.append((score[art],names[art]))
    # in the end we have a list of the articles sorted according to the block ranking vector and the articles score
    ranking.extend([x[1] for x in sorted(s,reverse=True)])

In [15]:
#results printing
for i in ranking[:50]:
    print(i)

John Wayne
Clint Eastwood
Elvis Presley
Woody Allen
Frank Sinatra
Steven Spielberg
Marilyn Monroe
Henry Fonda
Lucille Ball
Bette Davis
Katharine Hepburn
Cary Grant
Dean Martin
Joan Crawford
Humphrey Bogart
Al Pacino
Robert De Niro
Martin Scorsese
Marlon Brando
Tom Hanks
Jack Nicholson
Bing Crosby
Ronald Reagan
Cecil B. DeMille
Orson Welles
Adam Sandler
Paul Newman
Mickey Rooney
Judy Garland
James Cagney
Burt Reynolds
Clark Gable
Charlton Heston
Arnold Schwarzenegger
Gary Cooper
Robin Williams
Tom Cruise
Mary Pickford
Bruce Willis
Spencer Tracy
Ginger Rogers
Sean Penn
Madonna (entertainer)
Nicolas Cage
Jack Lemmon
Robert Redford
Steve Martin
Quentin Tarantino
Meryl Streep
Jerry Lewis
